In [1]:
class RBNode:

    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None
        self.parent = None
        self.colour = "R"

    def get_uncle(self):
        return

    def is_leaf(self):
        return self.left == None and self.right == None

    def is_left_child(self):
        return self == self.parent.left

    def is_right_child(self):
        return not self.is_left_child()

    def is_red(self):
        return self.colour == "R"

    def is_black(self):
        return not self.is_red()

    def make_black(self):
        self.colour = "B"

    def make_red(self):
        self.colour = "R"

    def get_brother(self):
        if self.parent.right == self:
            return self.parent.left
        return self.parent.right

    def get_uncle(self):
        return self.parent.get_brother()

    def uncle_is_black(self):
        if self.get_uncle() == None:
            return True
        return self.get_uncle().is_black()

    def __str__(self):
        return "(" + str(self.value) + "," + self.colour + ")"

    def __repr__(self):
         return "(" + str(self.value) + "," + self.colour + ")"

    def rotate_right(self):
        #TODO
        return 0

    def rotate_left(self):
        #TODO
        return 0
                    
        
    

In [2]:
class RBTree:

    def __init__(self,):
        self.root = None

    def is_empty(self,):
        return self.root == None

    def get_height(self,):
        if self.is_empty():
            return 0
        return self.__get_height(self.root)

    def __get_height(self, node):
        if node == None:
            return 0
        return 1 + max(self.__get_height(node.left), self.__get_height(node.right))

    def insert(self, value):
        if self.is_empty():
            self.root = RBNode(value)
            self.root.make_black()
        else:
            self.__insert(self.root, value)

    def __insert(self, node, value):
        if value < node.value:
            if node.left == None:
                node.left = RBNode(value)
                node.left.parent = node
                self.fix(node.left)
            else:
                self.__insert(node.left, value)
        else:
            if node.right == None:
                node.right = RBNode(value)
                node.right.parent = node
                self.fix(node.right)
            else:
                self.__insert(node.right, value)

    def fix(self, node):
        #You may alter code in this method if you wish, it's merely a guide.
        if node.parent == None:
            node.make_black()
        while node != None and node.parent != None and node.parent.is_red(): 
            #TODO
        self.root.make_black()

    def __str__(self):
        if self.is_empty():
            return "[]"
        return "[" + self.__str_helper(self.root) + "]"

    def __str_helper(self, node):
        if node.is_leaf():
            return "[" + str(node) + "]"
        if node.left == None:
            return "[" + str(node) + " -> " + self.__str_helper(node.right) + "]"
        if node.right == None:
            return "[" +  self.__str_helper(node.left) + " <- " + str(node) + "]"
        return "[" + self.__str_helper(node.left) + " <- " + str(node) + " -> " + self.__str_helper(node.right) + "]"


IndentationError: expected an indented block after 'while' statement on line 46 (87019639.py, line 48)

# Part 1: Implementing and Analyzing Red-Black Trees

In your 2C03 lectures, you have studied Red-Black Trees (RBT) and seen (at least in theory) how they are implemented. If you do not remember them or never studied them, now is the time. Given in the notebook is a partial implementation of Red-Black Trees with three methods `rotate_left()`, `rotate_right()`, and `fix()`. `rotate_left` performs the operation of rotating the node left and `rotate_right` performs the operation of rotating the node right. The nodes also point back to their parents, and the `fix` method updates the tree once the rotation has been made.


In this exercise, complete these implementations by enforcing the properties of Binary Search Trees that make it an RBT. Two of these properties are:

- Red nodes cannot have red children.
- All simple paths from the root to a leaf must contain the same number of black nodes.

Every time you insert an element in the tree, the above properties get violated and need to be fixed.


## Part 1.2
In this section, you will compare your implementation against traditional implementations. Take ALL the
Binary Search implementations we have discussed in the first few weeks and design an experiment to
compare the performance against your dynamic implementation.
In your reflection section, describe in detail the experiment design, what are the various list sizes and the
number of trials you chose. Also, describe your observations. Did you notice any performance
improvement? When does dynamic binary search outperform others? When is it an overkill? Describe in
detail your experiments design, and the observations.

## Part 1.3
In this experiment, generate a perfectly sorted list and record the difference between the height of the two
trees for multiple rounds. Then create different versions of the list varying the level of “unsortedness”
(like you have done in previous labs with near-sorted lists and unsorted lists). You can use controlled
quicksort to create these versions of near-sorted list. In your reflection, describe the experiment design,
and the runs, and show in the graph how the “degree of sortedness” impacts the height of the two trees.
Refer to the swap count experiment you ran to determine the degree of sortedness. Write a detailed
reflection on what you observe and why it might happen.

## Part 2: Making Binary Search Dynamic
Binary search of a sorted array takes logarithmic search time. However, if you notice closely, the
insertion of new element takes linear time. By keeping several sorted arrays, you can improve the time for
insertion. For instance, you can SEARCH and INSERT separately on a set of n values. It can go
something like this for a set of n elements

## Part 2.1
Implement the SEARCH, INSERT and DELETE operations for binary search using the above
description. For this, the code has not been provided. Feel free to go back to previous labs and use the
Binary Search implementations discussed there, to build on.

## Part 2.2

In this section, you will compare your implementation against traditional implementations. Take ALL the
Binary Search implementations we have discussed in the first few weeks and design an experiment to
compare the performance against your dynamic implementation.
In your reflection section, describe in detail the experiment design, what are the various list sizes and the
number of trials you chose. Also, describe your observations. Did you notice any performance
improvement? When does dynamic binary search outperform others? When is it an overkill? Describe in
detail your experiments design, and the observations.
